In [31]:
import requests
from datasets import load_dataset
from huggingface_hub import hf_api, ModelCard
from huggingface_hub.utils import EntryNotFoundError,RepositoryNotFoundError
import pandas as pd
import matplotlib.pyplot as plt
import rich
import random

In [2]:
api = hf_api.HfApi()

In [3]:
rich.inspect(api.list_models)

╭────────── <bound method HfApi.list_models of <huggingface_hub.hf_api.HfApi object at 0x7fd4d8231700>> ──────────╮
│ def HfApi.list_models(*, filter: 'Union[ModelFilter, str, Iterable[str], None]' = None, author: 'Optional[str]' │
│ = None, library: 'Optional[Union[str, List[str]]]' = None, language: 'Optional[Union[str, List[str]]]' = None,  │
│ model_name: 'Optional[str]' = None, task: 'Optional[Union[str, List[str]]]' = None, trained_dataset:            │
│ 'Optional[Union[str, List[str]]]' = None, tags: 'Optional[Union[str, List[str]]]' = None, search:               │
│ 'Optional[str]' = None, emissions_thresholds: 'Optional[Tuple[float, float]]' = None, sort:                     │
│ "Union[Literal['last_modified'], str, None]" = None, direction: 'Optional[Literal[-1]]' = None, limit:          │
│ 'Optional[int]' = None, full: 'Optional[bool]' = None, cardData: 'bool' = False, fetch_config: 'bool' = False,  │
│ token: 'Union[bool, str, None]' = None, pipeline_tag: 'Optional[str]' = None) -> 'Iterable[ModelInfo]':         │
│                                                                                                                 │
│ List models hosted on the Huggingface Hub, given some filters.                                                  │
│                                                                                                                 │
│ 28 attribute(s) not shown. Run inspect(inspect) for options.                                                    │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [4]:
hub_datasets = list(iter(api.list_datasets(limit=None, filter=None, full=True)))

In [5]:
hub_models = list(iter(api.list_models(limit=None, filter=None, full=True)))

In [6]:
hub_models[0]

ModelInfo(id='albert/albert-base-v1', author='albert', sha='082438ba120d36b97b9288772a41144e941705b9', created_at=datetime.datetime(2022, 3, 2, 23, 29, 4, tzinfo=datetime.timezone.utc), last_modified=datetime.datetime(2024, 2, 19, 10, 57, 35, tzinfo=datetime.timezone.utc), private=False, gated=False, disabled=None, downloads=12830, likes=7, library_name='transformers', tags=['transformers', 'pytorch', 'tf', 'safetensors', 'albert', 'fill-mask', 'exbert', 'en', 'dataset:bookcorpus', 'dataset:wikipedia', 'arxiv:1909.11942', 'license:apache-2.0', 'autotrain_compatible', 'endpoints_compatible', 'region:us'], pipeline_tag='fill-mask', mask_token=None, card_data=None, widget_data=None, model_index=None, config=None, transformers_info=None, siblings=[RepoSibling(rfilename='.gitattributes', size=None, blob_id=None, lfs=None), RepoSibling(rfilename='README.md', size=None, blob_id=None, lfs=None), RepoSibling(rfilename='config.json', size=None, blob_id=None, lfs=None), RepoSibling(rfilename='mod

In [7]:
hub_item_dict = []
for item in hub_models:
        data = item.__dict__
        hub_item_dict.append(data)

In [8]:
# df = pd.DataFrame.from_dict(hub_item_dict)
# df = load_dataset("librarian-bots/model_cards_with_metadata")['train'].to_pandas()

Generating train split:   0%|          | 0/752616 [00:00<?, ? examples/s]

In [9]:
df.columns

Index(['modelId', 'author', 'last_modified', 'downloads', 'likes',
       'library_name', 'tags', 'pipeline_tag', 'createdAt', 'card'],
      dtype='object')

In [10]:
df.loc[35, "tags"]

array(['transformers', 'pytorch', 'safetensors', 'canine',
       'feature-extraction', 'multilingual', 'af', 'sq', 'ar', 'an', 'hy',
       'ast', 'az', 'ba', 'eu', 'bar', 'be', 'bn', 'inc', 'bs', 'br',
       'bg', 'my', 'ca', 'ceb', 'ce', 'zh', 'cv', 'hr', 'cs', 'da', 'nl',
       'en', 'et', 'fi', 'fr', 'gl', 'ka', 'de', 'el', 'gu', 'ht', 'he',
       'hi', 'hu', 'is', 'io', 'id', 'ga', 'it', 'ja', 'jv', 'kn', 'kk',
       'ky', 'ko', 'la', 'lv', 'lt', 'roa', 'nds', 'lm', 'mk', 'mg', 'ms',
       'ml', 'mr', 'mn', 'min', 'ne', 'new', 'nb', 'nn', 'oc', 'fa',
       'pms', 'pl', 'pt', 'pa', 'ro', 'ru', 'sco', 'sr', 'scn', 'sk',
       'sl', 'aze', 'es', 'su', 'sw', 'sv', 'tl', 'tg', 'th', 'ta', 'tt',
       'te', 'tr', 'uk', 'ud', 'uz', 'vi', 'vo', 'war', 'cy', 'fry',
       'pnb', 'yo', 'dataset:bookcorpus', 'dataset:wikipedia',
       'arxiv:2103.06874', 'license:apache-2.0', 'endpoints_compatible',
       'region:us'], dtype=object)

In [11]:
#Check that the labels are inside the readme file
print(df.loc[35,"card"])

---
language: 
- multilingual
- af
- sq
- ar
- an
- hy
- ast
- az
- ba
- eu
- bar
- be
- bn
- inc
- bs
- br
- bg
- my
- ca
- ceb
- ce
- zh
- cv
- hr
- cs
- da
- nl
- en
- et
- fi
- fr
- gl
- ka
- de
- el
- gu
- ht
- he
- hi
- hu
- is
- io
- id
- ga
- it
- ja
- jv
- kn
- kk
- ky
- ko
- la
- lv
- lt
- roa
- nds
- lm
- mk
- mg
- ms
- ml
- mr
- mn
- min
- ne
- new
- nb
- nn
- oc
- fa
- pms
- pl
- pt
- pa
- ro
- ru
- sco
- sr
- hr
- scn
- sk
- sl
- aze
- es
- su
- sw
- sv
- tl
- tg
- th
- ta
- tt
- te
- tr
- uk
- ud
- uz
- vi
- vo
- war
- cy
- fry
- pnb
- yo
license: apache-2.0
datasets:
- bookcorpus
- wikipedia
---

# CANINE-c (CANINE pre-trained with autoregressive character loss) 

Pretrained CANINE model on 104 languages using a masked language modeling (MLM) objective. It was introduced in the paper [CANINE: Pre-training an Efficient Tokenization-Free Encoder for Language Representation](https://arxiv.org/abs/2103.06874) and first released in [this repository](https://github.com/google

In [12]:
card = ModelCard.load(df.loc[35,"modelId"]).data.to_dict()
print(card)

{'datasets': ['bookcorpus', 'wikipedia'], 'language': ['multilingual', 'af', 'sq', 'ar', 'an', 'hy', 'ast', 'az', 'ba', 'eu', 'bar', 'be', 'bn', 'inc', 'bs', 'br', 'bg', 'my', 'ca', 'ceb', 'ce', 'zh', 'cv', 'hr', 'cs', 'da', 'nl', 'en', 'et', 'fi', 'fr', 'gl', 'ka', 'de', 'el', 'gu', 'ht', 'he', 'hi', 'hu', 'is', 'io', 'id', 'ga', 'it', 'ja', 'jv', 'kn', 'kk', 'ky', 'ko', 'la', 'lv', 'lt', 'roa', 'nds', 'lm', 'mk', 'mg', 'ms', 'ml', 'mr', 'mn', 'min', 'ne', 'new', 'nb', 'nn', 'oc', 'fa', 'pms', 'pl', 'pt', 'pa', 'ro', 'ru', 'sco', 'sr', 'hr', 'scn', 'sk', 'sl', 'aze', 'es', 'su', 'sw', 'sv', 'tl', 'tg', 'th', 'ta', 'tt', 'te', 'tr', 'uk', 'ud', 'uz', 'vi', 'vo', 'war', 'cy', 'fry', 'pnb', 'yo'], 'license': 'apache-2.0'}


In [13]:
#Go through the card dictionary
for item in card.items():
    if type(item[1]) is list:
        for i in item[1]:
            print(i)
    else:
        print(item[1])

bookcorpus
wikipedia
multilingual
af
sq
ar
an
hy
ast
az
ba
eu
bar
be
bn
inc
bs
br
bg
my
ca
ceb
ce
zh
cv
hr
cs
da
nl
en
et
fi
fr
gl
ka
de
el
gu
ht
he
hi
hu
is
io
id
ga
it
ja
jv
kn
kk
ky
ko
la
lv
lt
roa
nds
lm
mk
mg
ms
ml
mr
mn
min
ne
new
nb
nn
oc
fa
pms
pl
pt
pa
ro
ru
sco
sr
hr
scn
sk
sl
aze
es
su
sw
sv
tl
tg
th
ta
tt
te
tr
uk
ud
uz
vi
vo
war
cy
fry
pnb
yo
apache-2.0


In [14]:
# create a list of random integers between 0 and 100000
idx_list = [random.randint(0, 100000) for i in range(10)]
# idx_list = [1]

for id in idx_list:
    dump_tags = df.loc[id, "tags"]
    # yml_tags = []
    try:
        card = str(ModelCard.load(df.loc[id,"modelId"]).data)
    except EntryNotFoundError:
        print(f"Error loading model card, no model card found for {df.loc[id, 'modelId']}")
        
    tags_not_in_card = []
    for tag in dump_tags: 
        if ":" in tag:
            tag = tag.split(":")[1]
        if tag not in card:
            all_dump_tags = False
            tags_not_in_card.append(tag)
    print(f"ModelId: {df.loc[id, 'modelId']}")
    print("Tags not in card: \n",tags_not_in_card)
    print("\nCard: \n",card)
        
    # print(card)
    

README.md:   0%|          | 0.00/234 [00:00<?, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


ModelId: Ching/negation_detector
Tags not in card: 
 ['transformers', 'pytorch', 'roberta', 'question-answering', 'endpoints_compatible', 'us']

Card: 
 {}


README.md:   0%|          | 0.00/163 [00:00<?, ?B/s]

ModelId: FreedomIntelligence/HuatuoGPT-7B
Tags not in card: 
 ['transformers', 'pytorch', 'baichuan', 'text-generation', 'custom_code', 'autotrain_compatible', 'us']

Card: 
 license: apache-2.0


...

ModelId: rkcr7/rkcrk
Tags not in card: 
 ['diffusers', '10.57967/hf/0273', 'autotrain_compatible', 'endpoints_compatible', 'StableDiffusionPipeline']

Card: 
 license: creativeml-openrail-m
tags:
- text-to-image
- stable-diffusion


README.md:   0%|          | 0.00/1.45k [00:00<?, ?B/s]

ModelId: as-cle-bert/tinyllama-essay-scorer
Tags not in card: 
 ['transformers', 'tensorboard', 'safetensors', 'text-generation', 'conversational', 'autotrain_compatible', 'text-generation-inference', 'endpoints_compatible', 'us']

Card: 
 base_model: TinyLlama/TinyLlama-1.1B-Chat-v1.0
license: apache-2.0
tags:
- generated_from_trainer
model-index:
- name: tinyllama-essay-scorer
  results: []


README.md:   0%|          | 0.00/304 [00:00<?, ?B/s]

ModelId: DanielClough/Candle_Mistral-7B-v0.1
Tags not in card: 
 ['transformers', 'gguf', 'autotrain_compatible', 'text-generation-inference', 'endpoints_compatible', 'us']

Card: 
 datasets:
- mistralai/Mistral-7B-v0.1
language:
- en
license: apache-2.0
pipeline_tag: text-generation


In [15]:
# Going throug the tag types

response = requests.get(
  "https://huggingface.co/api/models-tags-by-type",
  params={},
  headers={}
)
tags = response.json()

In [16]:
print(rich.inspect(tags))

╭─────────────────────────────────────────────── <class 'dict'> ───────────────────────────────────────────────╮
│ dict() -> new empty dictionary                                                                               │
│ dict(mapping) -> new dictionary initialized from a mapping object's                                          │
│     (key, value) pairs                                                                                       │
│ dict(iterable) -> new dictionary initialized as if via:                                                      │
│     d = {}                                                                                                   │
│     for k, v in iterable:                                                                                    │
│         d[k] = v                                                                                             │
│ dict(**kwargs) -> new dictionary initialized with the name=value pairs                                       │
│     in the keyword argument list.  For example:  dict(one=1, two=2)                                          │
│                                                                                                              │
│ ╭──────────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ {                                                                                                        │ │
│ │ │   'region': [                                                                                          │ │
│ │ │   │   {'type': 'region', 'label': '🇺🇸 Region: US', 'id': 'region:us'},                                 │ │
│ │ │   │   {'type': 'region', 'label': '🇪🇺 Region: EU', 'id': 'region:eu'}                                  │ │
│ │ │   ],                                                                                                   │ │
│ │ │   'other': [                                                                                           │ │
│ │ │   │   {'id': 'endpoints_compatible', 'label': 'Inference Endpoints', 'type': 'other'},                 │ │
│ │ │   │   {'id': 'autotrain_compatible', 'label': 'AutoTrain Compatible', 'type': 'other'},                │ │
│ │ │   │   {'id': 'text-generation-inference', 'label': 'text-generation-inference', 'type': 'other'},      │ │
│ │ │   │   {'id': 'model-index', 'label': 'Eval Results', 'type': 'other'},                                 │ │
│ │ │   │   {'id': '4-bit', 'label': '4-bit precision', 'type': 'other'},                                    │ │
│ │ │   │   {'id': 'merge', 'label': 'Merge', 'type': 'other'},                                              │ │
│ │ │   │   {'id': 'text-embeddings-inference', 'label': 'text-embeddings-inference', 'type': 'other'},      │ │
│ │ │   │   {'id': 'custom_code', 'label': 'custom_code', 'type': 'other'},                                  │ │
│ │ │   │   {'id': '8-bit', 'label': '8-bit precision', 'type': 'other'},                                    │ │
│ │ │   │   {'id': 'co2_eq_emissions', 'label': 'Carbon Emissions', 'type': 'other'},                        │ │
│ │ │   │   ... +1                                                                                           │ │
│ │ │   ],                                                                                                   │ │
│ │ │   'library': [                                                                                         │ │
│ │ │   │   {'id': 'pytorch', 'label': 'PyTorch', 'type': 'library'},                                        │ │
│ │ │   │   {'id': 'tf', 'label': 'TensorFlow', 'type': 'library'},                                          │ │
│ │ │   │   {'id': 'jax', 'label': 'JAX', 'type': 'library'},                                                │ │
│ │ │   │   {'id': 'transformers', 'label': 'Transformers', 'type': 'library'},                              │ │
│ │ │   │   {'id': 'safetensors', 'label': 'S

None


### Which tags can appear without being mentioned in the YAML section?

This can show us the list of tags that will appear if there is content related to the tag in the rest of the readme file or the project files. The interesting thing about this tags is that the HF platform has to be getting the information of this tags from somewhere.

In [30]:
idx_list = [random.randint(0, 720000) for i in range(10000)]
# idx_list = [1]
independent_tags = {}
for id in idx_list:
    dump_tags = df.loc[id, "tags"]
    # yml_tags = []
    try:
        card = str(ModelCard.load(df.loc[id,"modelId"],ignore_metadata_errors=True).data)
    except (EntryNotFoundError,RepositoryNotFoundError):
        print(f"Error loading model card, no model card found for {df.loc[id, 'modelId']}")
    # except (ValueError) as error:
    #     print(f"Error loading model card, wrong metadata card format for {df.loc[id, 'modelId']}")
        
    tags_not_in_card = []
    for tag in dump_tags: 
        if ":" in tag:
            tag = tag.split(":")[1]
        if tag not in card:
            all_dump_tags = False
            
            if tag not in independent_tags:
                independent_tags[tag] = 1
            else:
                independent_tags[tag] += 1
    # print(f"ModelId: {df.loc[id, 'modelId']}")
    # print("Tags not in card: \n",tags_not_in_card)
    # print("\nCard: \n",card)

README.md:   0%|          | 0.00/2.21k [00:00<?, ?B/s]

Error loading model card, no model card found for Ambrosepp/savio


README.md:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

Error loading model card, no model card found for araara11/nana
Error loading model card, no model card found for jason1i/bloom-3b-squad2
Error loading model card, no model card found for ericyu/MaskCD_SYSU_CD
Error loading model card, no model card found for patrickvonplaten/rag-sequence-gen-prev
Error loading model card, no model card found for nzabzzd/something
Error loading model card, no model card found for mllm-dev/yelp_finetuned_sbatch_upload_test
Error loading model card, no model card found for kashikashi/YosukeKashiwagi_librilight-6k-vad20s_ssl


...

README.md:   0%|          | 0.00/856 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/1.70k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

Error loading model card, no model card found for patrickvonplaten/pndm_try


RepositoryNotFoundError: 401 Client Error. (Request ID: Root=1-668d1cc1-0dbf64c00f4144062a160d30;b7f91d59-e7da-4e60-9f3a-518de83bfb3e)

Repository Not Found for url: https://huggingface.co/seongs/0621_v1.2/resolve/main/README.md.
Please make sure you specified the correct `repo_id` and `repo_type`.
If you are trying to access a private or gated repo, make sure you are authenticated.
Invalid username or password.

In [36]:
sorted_independent_tags = sorted(independent_tags.items(), key=lambda x: x[1])
print(rich.inspect(sorted_independent_tags))
print(len(independent_tags))

╭─────────────────────── <class 'list'> ───────────────────────╮
│ Built-in mutable sequence.                                   │
│                                                              │
│ ╭──────────────────────────────────────────────────────────╮ │
│ │ [                                                        │ │
│ │ │   ('mgpt', 1),                                         │ │
│ │ │   ('fastspeech2_conformer', 1),                        │ │
│ │ │   ('2010.13956', 1),                                   │ │
│ │ │   ('coreml', 1),                                       │ │
│ │ │   ('2207.12598', 1),                                   │ │
│ │ │   ('2205.11487', 1),                                   │ │
│ │ │   ('2406.15888', 1),                                   │ │
│ │ │   ('2105.09680', 1),                                   │ │
│ │ │   ('2402.13228', 1),                                   │ │
│ │ │   ('2108.08877', 1),                                   │ │
│ │ │   ... +609                                             │ │
│ │ ]                                                        │ │
│ ╰──────────────────────────────────────────────────────────╯ │
│                                                              │
│ 35 attribute(s) not shown. Run inspect(inspect) for options. │
╰──────────────────────────────────────────────────────────────╯

None
619


In [37]:
from datetime import datetime
import json

# Get the current time
current_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

# Create a filename with the current time
filename = f"sorted_independent_tags_{current_time}.json"

# Save the sorted dictionary as a JSON file
with open(filename, "w") as f:
    json.dump(dict(sorted_independent_tags), f, indent=4)

Finding the missing examples